# Aprendizaje por Transferencia (Transfer Learning)

La idea general del aprendizaje por transferencia es usar el conocimiento aprendido de las tareas para las cuales hay una gran cantidad de datos etiquetados disponibles en entornos donde solo hay pocos datos etiquetados disponibles. La creación de datos etiquetados es costosa, por lo que es iportante aprovechar al máximo los conjuntos de datos existentes.

En un modelo tradicional de aprendizaje automático, el objetivo principal es generalizar sobre datos aún no vistos, basado en patrones aprendidos de los datos de entrenamiento. Con el aprendizaje por transferencia, se intenta iniciar este proceso de generalización partiendo de patrones que se han aprendido para una tarea diferente. Esencialmente, en lugar de comenzar el proceso de aprendizaje desde una hoja en blanco (a menudo inicializada aleatoriamente), se parte de patrones que se han aprendido para resolver una tarea diferente.

### Ejemplo de Aprendizaje por Transferencia con el Conjunto de Datos CIFAR-10

Usaremos un conjunto de datos estandarizado llamado CIFAR-10. CIFAR-10 consta de 60000 imágenes. Hay 10 categorías diferentes y 6000 imágenes por categoría. Cada imagen tiene un tamaño de solo 32 por 32 píxeles.

<img src="figuras/CIFAR-10_examples.png" width="75%">

#### Modelos Pre-entrenados en Keras

Modelos disponibles en Keras (https://keras.io/applications/):
- Xception
- VGG16
- VGG19
- ResNet50
- InceptionV3
- InceptionResNetV2
- MobileNet
- DenseNet
- NASNet

En este ejemplo utilizaremos el modelo InceptionV3:
    
 <img src="figuras/inception.png" width="100%">  

#### Importar keras, CIFAR-10 e InceptionV3

In [1]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import scipy
from scipy import misc
import os

/Users/wladimir/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Cargar los datos de CIFAR-10

In [2]:
# cargar los datos
(x_entrenamiento, y_entrenamiento), (x_prueba, y_prueba) = cifar10.load_data()
y_entrenamiento = np.squeeze(y_entrenamiento)
print('datos cargados')

datos cargados


#### Cargar el modelo InceptionV3 y remover las capas finales de clasificación

In [3]:
# cargar modelo inceptionV3
modelo = InceptionV3(weights='imagenet', include_top=False, input_shape=(139, 139, 3))
print('modelo cargado')

modelo cargado


#### Obtener los atributos del conjunto de entrenamiento

Este nuevo modelo ya no devolverá la predicción de la clase de la imagen, ya que la capa de clasificación se ha eliminado; sin embargo, la Red Neuronal Convolucional ahora almacenada en el modelo todavía nos proporciona una forma útil de extraer los atributos de las imágenes. Al pasar cada una de las imágenes CIFAR-10 a través de este modelo, podemos convertir cada imagen de su matriz de 32x32x3 de píxeles a un vector con 2048 entradas. En la práctica, nos referimos a este conjunto de datos de puntos de 2048 dimensiones como los atributos de cuello de botella de InceptionV3.

Los atributos se almacerarán en un archivo de manera que subsiguientes corridas del código no tenga que volverlos a procesar.

In [4]:
# obtener atributos (entrenamiento)
if os.path.exists('inception_atributos_entrenamiento.npz'):
    print('cargar atributos del archivo de atributos existente (entrenamiento)')
    atributos = np.load('inception_atributos_entrenamiento.npz')['atributos']
else:
    print('no hay archivo de atributos existente (entrenamiento)')
    print('calculando ...')
    # preprocesar los datos de entrenamiento
    grande_x_entrenamiento = np.array([scipy.misc.imresize(x_entrenamiento[i], (139, 139, 3)) 
                            for i in range(0, len(x_entrenamiento))]).astype('float32')
    inception_entrada_entrenamiento = preprocess_input(grande_x_entrenamiento)
    print('datos de entrenamiento preprocesados')
    # extraer, procesar, y almacenar los atributos
    atributos = modelo.predict(inception_entrada_entrenamiento)
    atributos = np.squeeze(atributos)
    np.savez('inception_atributos_entrenamiento', atributos=atributos)
print('atributos almacenados (entrenamiento)')

cargar atributos del archivo de atributos existente (entrenamiento)
atributos almacenados (entrenamiento)


#### Obtener los atributos del conjunto de prueba

Se repite el mismo procedimiento para los datos de prueba

In [5]:
# obtener atributos (prueba)
if os.path.exists('inception_atributos_prueba.npz'):
    print('cargar atributos del archivo de atributos existente (prueba)')
    atributos_prueba = np.load('inception_atributos_prueba.npz')['atributos_prueba']
else:
    print('no hay archivo de atributos existente (prueba)')
    print('calculando ...')
    # preprocesar los datos de prueba
    grande_x_prueba = np.array([scipy.misc.imresize(x_prueba[i], (139, 139, 3)) 
                            for i in range(0, len(x_prueba))]).astype('float32')
    inception_entrada_prueba = preprocess_input(grande_x_prueba)
    print('datos de prueba preprocesados')
    # extraer, procesar, y almacenar los atributos (prueba)
    atributos_prueba = modelo.predict(inception_entrada_prueba)
    atributos_prueba = np.squeeze(atributos_prueba)
    np.savez('inception_atributos_prueba', atributos_prueba=atributos_prueba)
print('atributos almacenados (prueba)')

cargar atributos del archivo de atributos existente (prueba)
atributos almacenados (entrenamiento)


## Entrenar una Red Neuronal sencilla

#### Codificar etiquetas de clases usando codificación one-hot

In [7]:
from keras.utils import np_utils

# one-hot encode the labels
y_entrenamiento = np_utils.to_categorical(y_entrenamiento, 10)
y_prueba = np_utils.to_categorical(y_prueba, 10)

#### Crear Red Neuronal usando Keras

In [8]:
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D

modelo = Sequential()
modelo.add(Conv2D(filters=100, kernel_size=2, input_shape=atributos.shape[1:]))
modelo.add(Dropout(0.4))
modelo.add(GlobalAveragePooling2D())
modelo.add(Dropout(0.3))
modelo.add(Dense(10, activation='softmax'))
modelo.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_95 (Conv2D)           (None, 2, 2, 100)         819300    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 100)         0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 100)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 820,310
Trainable params: 820,310
Non-trainable params: 0
_________________________________________________________________


#### Compilar y entrenar modelo

In [9]:
modelo.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='modelo.mejor.hdf5', 
                               verbose=1, save_best_only=True)
modelo.fit(atributos, y_entrenamiento, batch_size=50, epochs=50,
          validation_split=0.2, callbacks=[checkpointer],
          verbose=2, shuffle=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 40000 samples, validate on 10000 samples
Epoch 1/50
Epoch 00001: val_loss improved from inf to 3.23936, saving model to modelo.mejor.hdf5
 - 47s - loss: 4.3853 - acc: 0.6828 - val_loss: 3.2394 - val_acc: 0.7695
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 33s - loss: 3.3390 - acc: 0.7657 - val_loss: 3.3096 - val_acc: 0.7709
Epoch 3/50
Epoch 00003: val_loss improved from 3.23936 to 2.74615, saving model to modelo.mejor.hdf5
 - 31s - loss: 3.2314 - acc: 0.7784 - val_loss: 2.7461 - val_acc: 0.8093
Epoch 4/50
Epoch 00004: val_loss did not improve
 - 21s - loss: 3.0612 - acc: 0.7905 - val_loss: 3.0611 - val_acc: 0.7947
Epoch 5/50
Epoch 00005: val_loss did not improve
 - 20s - loss: 3.0647 - acc: 0.7939 - val_loss: 3.0429 - val_acc: 0.7971
Epoch 6/50
Epoch 00006: val_loss did not improve
 - 21s - loss: 2.9063 - acc: 0.8049 - val_loss: 3.0748 - val_acc: 0.7956
Epoch 7/50
Epoch 00007: val_loss did not impr

#### Exactitud del modelo con el conjunto de prueba

In [10]:
# cargar los pesos que generaron la mejor exactitud sobre el conjunto de validación 
modelo.load_weights('modelo.mejor.hdf5')

# evaluar la exactitud sobre conjunto de prueba
score = modelo.evaluate(atributos_prueba, y_prueba, verbose=0)
exactitud = 100 * score[1]

# imprimir exactitud del conjunto de prueba
print('Exactitud prueba: %.4f%%' % exactitud)

Exactitud prueba: 82.4600%
